In [1]:
# Jedi not working
# %config Completer.use_jedi = True
# %config IPCompleter.greedy=True
import sys
import os
import argparse

import subprocess
import psycopg2 as pg
import pandas.io.sql as psql

import pandas as pd
import numpy as np
from io import StringIO
from dateutil import tz
from datetime import datetime, timedelta
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist, pdist
from yellowbrick.cluster import KElbowVisualizer
from sklearn.metrics import pairwise_distances_argmin_min

In [8]:
# Load Hourly Data
def get_hourly_data():
    try:
        connection = pg.connect("host=localhost dbname=traffic user=postgres password=1234")
        df = psql.read_sql("select * from public.hourly;",connection)
        return df
    except (Exception, pg.Error) as error:
        print("Error while fetching data from PostgreSQL", error)
    finally:
        # closing database connection.
        if connection:
            connection.close()
            print("PostgreSQL connection is closed")
segment_cluster_df = get_hourly_data()
segment_cluster_df = orig_df[orig_df.cluster_num.notna()]
segment_cluster_df.head()

PostgreSQL connection is closed


,osm_way_id,hour0,hour1,hour2,hour3,hour4,hour5,hour6,hour7,hour8,...,std_dev_15,std_dev_16,std_dev_17,std_dev_18,std_dev_19,std_dev_20,std_dev_21,std_dev_22,std_dev_23,cluster_num
1279,76967085.0,16.460000,26.179000,16.508000,19.465625,16.666833,12.724000,11.476458,12.812556,11.217077,...,8.424870,8.933565,3.055615,3.616975,3.754957,2.308271,7.214989,2.157000,16.919758,0.0
2716,4716859.0,26.305741,25.349394,25.255238,24.821519,24.136204,23.589649,22.908557,22.811266,22.948332,...,14.825014,14.554039,14.431588,14.533345,13.927472,14.683171,14.028156,13.956483,14.539765,0.0
2717,4716860.0,35.260313,37.106141,37.652427,37.477883,37.337509,36.999126,37.057193,36.192891,35.200139,...,10.872726,10.370045,10.005972,9.198437,8.720829,9.903083,9.918452,9.985960,10.463367,2.0
2718,4716862.0,37.565158,36.202676,35.867714,34.696579,34.250949,35.081571,34.759567,33.992688,33.088739,...,13.880063,13.738261,13.271994,13.947260,13.387598,13.500991,13.500429,13.841592,14.692277,2.0
2719,4716873.0,24.605367,24.489386,24.361795,23.581164,22.682313,22.363853,22.090231,22.460020,22.977684,...,15.071607,15.042599,15.390938,14.623581,14.565046,13.803390,14.112008,13.907971,14.274844,0.0


In [6]:
# Get sudden traffic stats 
def get_hourly_stats():
    try:
        connection = pg.connect("host=localhost dbname=traffic user=postgres password=1234")
        # psql.read_sql("set search_path to public;",connection)
        sqlQuery = """
            WITH A AS
                (select public.my_hour_extract(utc_timestamp) as hour_num,
                        count(case when sudden_traffic then 1 end) as sudden_traffic_count,
                        count(*) as total 
                 from public.speed 
                 group by public.my_hour_extract(utc_timestamp)),
              B AS
                (select public.my_hour_extract(utc_timestamp) as hour_num,
                        avg(consecutive_hours) as avg_hours_in_traffic
                 from public.speed 
                 where sudden_traffic
                 group by public.my_hour_extract(utc_timestamp))
            SELECT *
            FROM A,B
            WHERE A.hour_num = B.hour_num"""
        df = psql.read_sql(sqlQuery,connection)
        return df
    except (Exception, pg.Error) as error:
        print("Error while fetching data from PostgreSQL", error)
    finally:
        # closing database connection.
        if connection:
            connection.close()
            print("PostgreSQL connection is closed")
hourly_stats = get_hourly_stats()
hourly_stats['percentage'] = hourly_stats.apply(lambda row: (row.sudden_traffic_count*100.0)/row.total, axis=1)
hourly_stats.sort_values(by='percentage')

PostgreSQL connection is closed


,hour_num,sudden_traffic_count,total,hour_num,avg_hours_in_traffic,percentage
12,19,708,324370,19,3.000000,0.218269
18,18,1022,466958,18,1.556751,0.218863
22,17,1542,633566,17,1.376135,0.243384
23,16,2347,826715,16,1.290158,0.283895
6,15,3137,1023448,15,1.240038,0.306513
8,14,4295,1165004,14,1.235623,0.368668
7,20,1055,269356,20,4.181991,0.391675
1,13,5372,1238319,13,1.343075,0.433814
2,12,7102,1266485,12,1.421008,0.560765
4,4,10361,1232747,4,2.084934,0.840481


In [10]:
# Get sudden traffic stats  for segments
def get_segment_stats():
    try:
        connection = pg.connect("host=localhost dbname=traffic user=postgres password=1234")
        # psql.read_sql("set search_path to public;",connection)
        sqlQuery = """
            WITH A AS
                (select osm_way_id,
                        count(case when sudden_traffic then 1 end) as sudden_traffic_count,
                        count(*) as total 
                 from public.speed 
                 group by osm_way_id),
              B AS
                (select osm_way_id,
                        avg(consecutive_hours) as avg_hours_in_traffic
                 from public.speed 
                 where sudden_traffic
                 group by osm_way_id)
            SELECT *
            FROM A,B
            WHERE A.osm_way_id = B.osm_way_id;"""
        df = psql.read_sql(sqlQuery,connection)
        return df
    except (Exception, pg.Error) as error:
        print("Error while fetching data from PostgreSQL", error)
    finally:
        # closing database connection.
        if connection:
            connection.close()
            print("PostgreSQL connection is closed")
segment_stats = get_segment_stats()
segment_stats['percentage'] = segment_stats.apply(lambda row: (row.sudden_traffic_count*100.0)/row.total, axis=1)
segment_stats.sort_values(by='percentage')

PostgreSQL connection is closed


,osm_way_id,sudden_traffic_count,total,osm_way_id,avg_hours_in_traffic,percentage
1382,337522383.0,1,16788,337522383.0,1.000000,0.005957
1471,388388731.0,1,16235,388388731.0,1.000000,0.006160
2081,658575802.0,1,13976,658575802.0,1.000000,0.007155
931,184369608.0,1,13786,184369608.0,1.000000,0.007254
865,161487291.0,1,13389,161487291.0,1.000000,0.007469
...,...,...,...,...,...,...
675,56397052.0,1,10,56397052.0,1.000000,10.000000
933,184369652.0,1438,14104,184369652.0,1.473574,10.195689
824,126623663.0,1,9,126623663.0,1.000000,11.111111
1116,238178364.0,1,8,238178364.0,1.000000,12.500000


In [12]:
# Get sudden traffic stats 
def get_cluster_stats():
    try:
        connection = pg.connect("host=localhost dbname=traffic user=postgres password=1234")
        # psql.read_sql("set search_path to public;",connection)
        sqlQuery = """
            WITH A AS
                (select cluster_num,
                        COUNT( case when s.sudden_traffic then 1 end) as sudden_traffic_count,
                        count(*) as total
                 from public.speed s,
                      public.hourly h
                 where s.osm_way_id = h.osm_way_id
                 and h.cluster_num is not null
                 group by h.cluster_num),
              B AS
                (select cluster_num,
                        avg(consecutive_hours) as avg_hours_in_traffic
                 from public.speed s,
                      public.hourly h
                 where s.osm_way_id = h.osm_way_id
                 and h.cluster_num is not null
                 AND sudden_traffic
                 group by h.cluster_num)
            SELECT *
            FROM A,B
            WHERE A.cluster_num = B.cluster_num;"""
        df = psql.read_sql(sqlQuery,connection)
        return df
    except (Exception, pg.Error) as error:
        print("Error while fetching data from PostgreSQL", error)
    finally:
        # closing database connection.
        if connection:
            connection.close()
            print("PostgreSQL connection is closed")
cluster_stats = get_cluster_stats()
cluster_stats['percentage'] = cluster_stats.apply(lambda row: (row.sudden_traffic_count*100.0)/row.total, axis=1)
cluster_stats.sort_values(by='percentage')

PostgreSQL connection is closed


,cluster_num,sudden_traffic_count,total,cluster_num,avg_hours_in_traffic,percentage
1,1.0,14791,3254876,1.0,1.701170,0.454426
2,2.0,89243,8535079,2.0,1.902401,1.045603
0,0.0,106883,9958140,0.0,2.801016,1.073323


In [15]:
segment_cluster_df.to_csv('segment_cluster_df.csv',index=False)
hourly_stats.to_csv('hourly_df.csv',index=False)
segment_stats.to_csv('segment_df.csv',index=False)
cluster_stats.to_csv('cluster_df.csv',index=False)
